In [0]:
from pyspark.sql import functions as F
from delta.tables import *

TABELA_ORIGEM = "v_credit.silver.tb_motivo"
TABELA_DESTINO = "v_credit.gold.dm_motivo"

### Transformacao Silver → Gold (Dimensao Motivo)
Le tb_motivo (Silver) e adiciona coluna derivada de negocio:
- **ds_potencial_automacao**: Classificacao baseada em ds_criticidade
  - Criticidade BAIXA → "Alto Potencial Automação" (ex: Consulta de Saldo)
  - Criticidade MEDIA → "Médio Potencial"
  - Criticidade ALTA → "Baixo (Requer Humano)" (ex: Fraude, Bloqueio)

Faz MERGE em dm_motivo (Gold).

**Importante**: Dimensao critica para identificar oportunidades de automacao e reducao de custos (29% de economia potencial).

### Configuracao
Define origem (Silver tb_motivo) e destino (Gold dm_motivo - dimensao do Star Schema).

In [ ]:
df_silver = spark.read.table(TABELA_ORIGEM)

df_gold = df_silver.select(
    F.col("cd_motivo"),
    F.col("ds_motivo"),
    F.col("ds_categoria"),
    F.col("ds_criticidade"),
    
    F.when(F.upper(F.col("ds_criticidade")).like("%BAIXA%"), "Alto Potencial Automação")
    .when(F.upper(F.col("ds_criticidade")).like("%MEDIA%"), "Médio Potencial")
    .otherwise("Baixo (Requer Humano)").alias("ds_potencial_automacao")
)

DeltaTable.forName(spark, TABELA_DESTINO).alias("t").merge(
    df_gold.alias("s"), "t.cd_motivo = s.cd_motivo"
).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

print(f"✅ Dimensão Motivo carregada!")